Reference issue: https://github.com/testspace-com/testspace-colab/issues/22

In [1]:
import pandas as pd
import util as ut

In [2]:
projectResultsFile="test.projectResults.json"

In [3]:
projectResults = ut.rd_details_cases(projectResultsFile)
spaceResults   = projectResults[0]['results']

In [4]:
spaceResultsDict          = {}
prev_spaceResultsDictItem = {}
for result in spaceResults:
    spaceResultsDict[result['name']] = ut.convert_to_tuples_dict(result['cases'], prev_spaceResultsDictItem)
    prev_spaceResultsDictItem        = spaceResultsDict[result['name']]

In [5]:
spaceDataset = pd.DataFrame(spaceResultsDict)
spaceDataset.sort_index(inplace=True)
spaceDataset.index.names = ['case', 'result']

In [6]:
spaceDataset = pd.DataFrame(spaceResultsDict)
spaceDataset.sort_index(inplace=True)
spaceDataset.index.names = ['case', 'result']

In [7]:
spaceDataset

R3       R2       R1
case result                          
c1   duration   1.0      1.0      1.0
     path        S1       S1       S1
     state      NEW  PASSING    FLAKY
     status       0        1        0
     track        1        2        1
c2   duration   2.0      2.0      2.0
     path        S1       S1       S1
     state     NONE     NONE     NONE
     status       1        1        1
     track        0        0        0
c3   duration   3.0      3.0      3.0
     path        S1       S1       S1
     state     NONE      NEW  PASSING
     status       1        0        1
     track        0        1        2
c4   duration   3.0      NaN      NaN
     path        S1      NaN      NaN
     state      NEW      NaN      NaN
     status       0      NaN      NaN
     track        1      NaN      NaN

In [8]:
cases      = spaceDataset.index.get_level_values("case").unique()
results    = list(spaceDataset.columns.values)
numResults = len(results)

In [9]:
spaceResultsSummaryArr = []
for case in cases:
    data = {}
    data['case']    = case
    caseStatus      = spaceDataset.loc[(case,"status")]
    data['count']   = caseStatus.count() ## returns number of non-NaN values in the column
    data['pr']      = "{:.0%}".format(caseStatus.sum()/caseStatus.count())
    data['time']    = round(spaceDataset.loc[(case, "duration")].mean(),2)
    data['history'] = spaceDataset.loc[(case)]
    spaceResultsSummaryArr.append(data)

In [10]:
spaceDatasetSummary = pd.DataFrame(spaceResultsSummaryArr)
spaceDatasetSummary.set_index('case', inplace=True)
spaceDatasetSummary = spaceDatasetSummary[['pr', 'count', 'time', 'history']]
spaceDatasetSummary.sort_index(inplace=True)

In [11]:
spaceDatasetSummary

,pr,count,time,history
case,,,,
c1,33%,3,1.0,R3 R2 R1 result ...
c2,100%,3,2.0,R3 R2 R1 result ...
c3,67%,3,3.0,R3 R2 R1 result ...
c4,0%,1,3.0,R3 R2 R1 result ...


In [12]:
spaceDatasetSummary.loc[('c1','history')].transpose()

result,duration,path,state,status,track
R3,1.0,S1,NEW,0,1
R2,1.0,S1,PASSING,1,2
R1,1.0,S1,FLAKY,0,1
